In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline

In [23]:
os.chdir('dataset/')
if os.path.isdir('train/healthy') is False:
    os.makedirs('train/healthy')
    os.makedirs('train/unhealthy')
    os.makedirs('valid/healthy')
    os.makedirs('valid/unhealthy')
    os.makedirs('test/healthy')
    os.makedirs('test/unhealthy')
    
    for c in random.sample(glob.glob('d*'), 21):
        shutil.move(c, 'train/unhealthy')
    for c in random.sample(glob.glob('h*'), 21):
        shutil.move(c, 'train/healthy')
    for c in random.sample(glob.glob('d*'), 5):
        shutil.move(c, 'valid/unhealthy')
    for c in random.sample(glob.glob('h*'), 5):
        shutil.move(c, 'valid/healthy')
    for c in random.sample(glob.glob('d*'), 2):
        shutil.move(c, 'test/unhealthy')
    for c in random.sample(glob.glob('h*'), 2):
        shutil.move(c, 'test/healthy')

In [24]:
os.chdir('../')

In [25]:
train_path = 'dataset/train'
valid_path = 'dataset/valid'
test_path = 'dataset/test'

In [26]:
train_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['unhealthy', 'healthy'], batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['unhealthy', 'healthy'], batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function= tf.keras.applications.vgg16.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['unhealthy', 'healthy'], batch_size=10, shuffle=False)


Found 42 images belonging to 2 classes.
Found 10 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [27]:
vgg16_model =tf.keras.applications.vgg16.VGG16()

In [28]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [29]:
for layer in model.layers:
    layer.trainable =False

In [30]:
model.add(Dense(units =2, activation= 'softmax'))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [32]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(x=train_batches, validation_data= valid_batches, epochs=5, verbose=2)

Epoch 1/5
5/5 - 12s - loss: 0.8188 - accuracy: 0.6190 - val_loss: 0.6584 - val_accuracy: 0.5000 - 12s/epoch - 2s/step
Epoch 2/5
5/5 - 10s - loss: 0.6125 - accuracy: 0.7143 - val_loss: 0.5306 - val_accuracy: 0.6000 - 10s/epoch - 2s/step
Epoch 3/5
5/5 - 10s - loss: 0.4679 - accuracy: 0.7857 - val_loss: 0.4606 - val_accuracy: 0.7000 - 10s/epoch - 2s/step
Epoch 4/5
5/5 - 10s - loss: 0.3650 - accuracy: 0.8095 - val_loss: 0.4163 - val_accuracy: 0.7000 - 10s/epoch - 2s/step
Epoch 5/5
5/5 - 10s - loss: 0.2897 - accuracy: 0.8810 - val_loss: 0.3669 - val_accuracy: 0.7000 - 10s/epoch - 2s/step


In [34]:
predictions = model.predict(x=test_batches, verbose=0)

In [35]:
test_batches.classes

array([0, 0, 1, 1])

In [36]:
from joblib import dump

In [37]:
dump(model, './../savedModel/model.joblib')

['./../savedModel/model.joblib']